# Assignment 1 - Autoregressive models with Transformers
## Deep Generative Modeling teaching material

#### Instructions on how to use this notebook:

This notebook is hosted on ``Google Colab``. To be able to work on it, you have to create your own copy. Go to *File* and select *Save a copy in Drive*.

You can also avoid using ``Colab`` entirely, and download the notebook to run it on your own machine. If you choose this, go to *File* and select *Download .ipynb*.

The advantage of using **Colab** is that you can use a GPU. You can complete this assignment with a CPU, but it will take a bit longer. Furthermore, we encourage you to train using the GPU not only for faster training, but also to get experience with this setting. This includes moving models and tensors to the GPU and back. This experience is very valuable because for various models and large datasets (like large CNNs for ImageNet, or Transformer models trained on Wikipedia), training on GPU is the only feasible way.

The default ``Colab`` runtime does not have a GPU. To change this, go to *Runtime - Change runtime type*, and select *GPU* as the hardware accelerator. The GPU that you get changes according to what resources are available at the time, and its memory can go from a 5GB, to around 18GB if you are lucky. If you are curious, you can run the following in a code cell to check:

```sh
!nvidia-smi
```

Note that despite the name, ``Google Colab`` does  not support collaborative work without issues. When two or more people edit the notebook concurrently, only one version will be saved. You can choose to do group programming with one person sharing the screen with the others, or make multiple copies of the notebook to work concurrently.

**Submission:** Please bring your (partial) solution to instruction sessions. Then you can discuss it with intructors and your colleagues.

In [ ]:
!nvidia-smi

## Introduction

In this assignment, we are going to implement an autoregressive model (ARM). An AMR is a likelihood-based deep generative model that utilizes the product rule and generates new object one-by-one. Transformers are current state-of-the-art architectures used for Large Language Models (LLMs). Specifically, generative LLMs are parameterized by so called decoder-transformers. The model used in this assignemnt is based on the architecture of so called Generative Pretrained Transformers (GPTs):
- [Radford, A., Narasimhan, K., Salimans, T. and Sutskever, I., 2018. Improving language understanding by generative pre-training.](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)

You can read more about ARMs in Chapter 2 of the following book:
- [Tomczak, J.M., "Deep Generative Modeling", Springer, 2022](https://link.springer.com/book/10.1007/978-3-030-93158-2)

You can read more about transformers in Chapter 12 of the following book:
- [Prince, S.J.D., "Understanding Deep Learning", MIT Press, 2023](https://udlbook.github.io/udlbook/)

In particular, the goals of this assignment are the following:

- Understand how transformer-based ARMs are formulated.
- Implement components of transformer-based ARMs using PyTorch.
- Train and evaluate a transformer-based ARM for text data.

This notebook is essential for preparing a report. Moreover, please remember to submit the final notebook together with the report (PDF).

### Theory behind ARMs

Let us consider a high-dimensional random variable $\mathbf{x} \in \mathcal{X}^{T}$ where $\mathcal{X} = \{0,1,\dots , L-1\}$ or $\mathcal{X} = \mathbb{R}$. Our goal is to model $p(\mathbf{x})$. We can apply the product rule to express this distribution as follows:
$$
p(\mathbf{x}) = p(x_1) \prod_{t=2}^{T} p(x_{t}|\mathbf{x}_{<t}) ,
$$
where $\mathbf{x}_{<t} = [x_1, x_2, \ldots , x_{t-1}]^{\top}$. For instance, for $\mathbf{x} = [x_1, x_2, x_{3}]^{\top}$, we have $p(\mathbf{x}) = p(x_1) p(x_{2}|x_{1}) p(x_{3} | x_{1}, x_{2})$.

The generative procedure is straightforward: We start with $x_1 \sim p(x_1)$, and then we proceed with $x_t \sim p(x_{t}|\mathbf{x}_{<t})$ by plugging in all previously sampled variables $\mathbf{x}_{<t}$. We can think of this procedure as a for-loop.

Now, the main goal is how to parameterize conditional distributions $p(x_{t}|\mathbf{x}_{<t})$. We can accomplish that by using neural networks, in particular, transformers. In this assignment, we focus on <i>decoder transformers</i> that utilize causal multi-head self-attention.

### Note

In this assignment, we build a simple LLM model. For this purpose, we use a dataset consisting of $\sim 8.5$k newspaper headlines, and each headline contain at most 150 letters (tokens). You are provided with a tokenizer for turning characters into a sequence of integers and padding, and text processing functions (e.g., removing special characters). Your model will be trained with 1.3M tokens per iteration, and will consist of few millions to over dozen millions of weights.

These numbers do not necessarilly impress anyone in the LLM community. However, please be aware that such datasets and models are not small and could be treated as a small-sized LLM-based problems. As you will notice in the end, we can still observe similar phenomena like hallucinations and the power of scaling up.

## IMPORTS

In [ ]:
# DO NOT REMOVE!
import os

import pickle

import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import numpy as np

!pip install datasets
from datasets import load_dataset

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

!pip install pytorch_model_summary
from pytorch_model_summary import summary

In [ ]:
# DO NOT REMOVE OR MODIFY
# Check if GPU is available and determine the device
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

print(f'The available device is {device}')

In [ ]:
# DO NOT REMOVE! (unless you work locally)
# mount drive: WE NEED IT FOR SAVING IMAGES! NECESSARY FOR GOOGLE COLAB!
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# DO NOT REMOVE! (unless you work locally)
# PLEASE CHANGE IT TO YOUR OWN GOOGLE DRIVE OR YOUR LOCAL DIR!
results_model_dir = '/content/gdrive/My Drive/Colab/Results/'

## Auxiliary classes and functions

Let us define some useful classes:
1. DataProcessor: "cleaning" texts.
2. Tokenizer: transforming characters to integers and padding.

In [ ]:
# DO NOT REMOVE OR MODIFY
class DataProcessor(object):
    def __init__(self, ):
        super().__init__()
        nlp = spacy.load("en_core_web_sm")
        nltk.download('omw-1.4')
        nltk.download("punkt")
        nltk.download("wordnet")
        nltk.download("stopwords")

    @staticmethod
    def preprocess_text(text):
        # Tokenize, remove punctuation and lowercase
        tokens = nltk.word_tokenize(text)
        tokens = [word.lower() for word in tokens if word.isalpha()]

        # Remove stopwords and lemmatize
        stop_words = set(stopwords.words("english"))
        lemmatizer = WordNetLemmatizer()
        processed_text = [
            lemmatizer.lemmatize(word) for word in tokens if word not in stop_words
        ]

        return " ".join(processed_text)

    def process_batch(self, texts):
        return [self.preprocess_text(d) for d in texts]

In [ ]:
# DO NOT REMOVE OR MODIFY
class Tokenizer(object):
    def __init__(self, max_length=0):
        super().__init__()

        self.max_length = max_length

        self.alphabet_letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

        self.alphabet = self.prepare_alphabet()
        self.decoded_alphabet = self.prepare_decoded_alphabet()

    def prepare_alphabet(self):
        # PREPARE THE ALPHABET (CHAR->INT)
        # as a dictionary
        alphabet = {}
        alphabet['pad'] = 0  # add 'pad'
        count = 1

        for letter in self.alphabet_letters:
            alphabet[letter] = count
            count += 1

        # add ' ', 'cls' tokens
        alphabet[' '] = count
        alphabet['cls'] = count + 1

        return alphabet

    def prepare_decoded_alphabet(self):
        # PREPARE DECODED ALPHABET (INT->CHAR)
        decoded_alphabet_ints = [i for i in range(len(self.alphabet_letters))]

        decoded_alphabet = {}
        decoded_alphabet[0] = 'pad'

        for i in decoded_alphabet_ints:
            decoded_alphabet[i+1] = self.alphabet_letters[i]

            decoded_alphabet[i+2] = ' '
        decoded_alphabet[i+3] = 'cls'

        return decoded_alphabet

    def encode(self, texts):
        N = len(texts)

        if self.max_length == 0:
            max_length = 0
            for i in range(N):
                len_i = len(texts[i])
                if len_i > max_length:
                    max_length = len_i
        else:
            max_length = self.max_length

        tokens = np.zeros((N, max_length+1))

        for i in range(N):
            len_i = len(texts[i])
            for j in range(-1, max_length):
                if j == -1:
                    tokens[i,j+1] = self.alphabet['cls']
                elif j >= len_i:
                    tokens[i,j+1] = self.alphabet['pad']
                else:
                    if texts[i][j] == 'é':
                        tokens[i,j+1] = self.alphabet['e']
                    elif texts[i][j] == 'í':
                        tokens[i,j+1] = self.alphabet['e']
                    elif texts[i][j] == 'á':
                        tokens[i,j+1] = self.alphabet['a']
                    elif texts[i][j] == 'ó':
                        tokens[i,j+1] = self.alphabet['o']
                    elif texts[i][j] == 'æ':
                        tokens[i,j+1] = self.alphabet['a']
                    elif texts[i][j] == 'ä':
                        tokens[i,j+1] = self.alphabet['a']
                    else:
                        tokens[i,j+1] = self.alphabet[texts[i][j]]

        return tokens

    def decode(self, tokens):
        texts = []

        for i in range(len(tokens)):
            tokens_i = tokens[i,:]
            text_i = ''
            for j in range(len(tokens_i)):
                if tokens_i[j] == 0:
                    break
                else:
                    if self.decoded_alphabet[tokens_i[j]] != 'cls':
                        text_i += self.decoded_alphabet[tokens_i[j]]
            texts.append(text_i)

        return texts

Some useful functions:

In [ ]:
# DO NOT REMOVE OR MODIFY
def save_texts(sampled_texts, name=''):
    # open file in write mode
    with open(results_dir + '/samples_' + name + '.txt', 'w') as fp:
        for item in sampled_texts:
            # write each item in a new line
            fp.write("%s\n" % item)

# Data

In [ ]:
# DO NOT REMOVE OR MODIFY
class Headers(Dataset):
    """A simple dataset based on headers. Source: https://huggingface.co/datasets/IlyaGusev/headline_cause"""

    def __init__(self, dataprocessor, tokenizer, mode='train', num_training_data=None, transforms=None):
        # LOAD DATA
        dataset = load_dataset("IlyaGusev/headline_cause", "en_simple")

        # PREPARE DATA
        if mode == 'train':
            train_texts = dataprocessor.process_batch(dataset['train'][:]['left_title'] + dataset['train'][:]['right_title']) # list
            if num_training_data is None:
                self.data = torch.from_numpy(tokenizer.encode(train_texts)).long()
            else:
                self.data = torch.from_numpy(tokenizer.encode(train_texts))[:num_training_data].long()
        elif mode == 'val':
            validation_texts = dataprocessor.process_batch(dataset['validation'][:]['left_title'] + dataset['validation'][:]['right_title']) # list
            self.data = torch.from_numpy(tokenizer.encode(validation_texts)).long()
        else:
            test_texts = dataprocessor.process_batch(dataset['test'][:]['left_title'] + dataset['test'][:]['right_title']) # list
            self.data = torch.from_numpy(tokenizer.encode(test_texts)).long()

        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transforms:
            sample = self.transforms(sample)
        return sample

## Implementing ARMs with Transformers


### Loss Function (NLL)
Our loss function is the negative log-likelihood for the categorical distribution (i.e., the cross-entropy loss).

Please note how it is implemented and how tokens (T) are handled.

In [ ]:
# DO NOT REMOVE OR MODIFY
class LossFun(nn.Module):
    def __init__(self,):
        super().__init__()

        self.loss = nn.NLLLoss(reduction='none')

    def forward(self, y_model, y_true, reduction='sum'):
        # y_model: B(atch) x T(okens) x V(alues)
        # y_true: B x T
        B, T, V = y_model.size()

        y_model = y_model.view(B * T, V)
        y_true = y_true.view(B * T,)

        loss_matrix = self.loss(y_model, y_true) # B*T

        if reduction == 'sum':
            return torch.sum(loss_matrix)
        elif reduction == 'mean':
            loss_matrix = loss_matrix.view(B, T)
            return torch.mean(torch.sum(loss_matrix, 1))
        else:
            raise ValueError('Reduction could be either `sum` or `mean`.')

### Transformer block

Transformers consist of transformer block. In the cell below, please define a transformer block.

In [ ]:
# YOUR CODE GOES HERE
# NOTE: The class must containt the following elements:
# (i) components (nn.Module) of a transformer bloc
# (ii) the forward function
# Moreover, forward must return the processed input

class TransformerBlock(nn.Module):
    def __init__(self, num_emb, num_neurons, num_heads=4):
        super().__init__()

        # hyperparams
        self.D = num_emb  # the embedding size
        self.H = num_heads  # the number of heads
        self.neurons = num_neurons  # the number of neurons in MLP

        # components

        #
        # your code goes here
        #
        # please define all necessary components that constitute a transformer block
        # NOTE: please use hyperparams defined above

    def forward(self, x, causal=True):
        #
        # your code goes here
        #
        # please define the forward pass for a tranformer block

### ARM (Decoder-Transformer)

Once we have a class for transformer blocks, we need to define a decoder-transformer that defines an auto-regressive model.

In [ ]:
# DO NOT REMOVE OR MODIFY
class DecoderTransformer(nn.Module):
    def __init__(self, num_tokens, num_token_vals, num_emb, num_neurons, num_heads=2, dropout_prob=0.1, num_blocks=10, device='cpu'):
        super().__init__()

        # hyperparams
        self.device = device
        self.num_tokens = num_tokens
        self.num_token_vals = num_token_vals
        self.num_emb = num_emb
        self.num_blocks = num_blocks

        # embedding layer
        self.embedding = torch.nn.Embedding(num_token_vals, num_emb)

        # positional embedding
        self.positional_embedding = nn.Embedding(num_tokens, num_emb)

        # transformer blocks
        self.transformer_blocks = nn.ModuleList()
        for _ in range(num_blocks):
            self.transformer_blocks.append(TransformerBlock(num_emb=num_emb, num_neurons=num_neurons, num_heads=num_heads))

        # output layer (logits + softmax)
        self.logits = nn.Sequential(nn.Linear(num_emb, num_token_vals))

        # dropout layer
        self.dropout = nn.Dropout(dropout_prob)

        # loss function
        self.loss_fun = LossFun()

    def transformer_forward(self, x, causal=True, temperature=1.0):
        # x: B(atch) x T(okens)
        # embedding of tokens
        x = self.embedding(x) # B x T x D
        # embedding of positions
        pos = torch.arange(0, x.shape[1], dtype=torch.long).unsqueeze(0).to(self.device)
        pos_emb = self.positional_embedding(pos)
        # dropout of embedding of inputs
        x = self.dropout(x + pos_emb)

        # transformer blocks
        for i in range(self.num_blocks):
            x = self.transformer_blocks[i](x)

        # output logits
        out = self.logits(x)

        return F.log_softmax(out/temperature, 2)

    @torch.no_grad()
    def sample(self, batch_size=4, temperature=1.0):
        x_seq = np.asarray([[self.num_token_vals - 1] for i in range(batch_size)])

        # sample next tokens
        for i in range(self.num_tokens-1):
            xx = torch.tensor(x_seq, dtype=torch.long, device=self.device)
            # process x and calculate log_softmax
            x_log_probs = self.transformer_forward(xx, temperature=temperature)
            # sample i-th tokens
            x_i_sample = torch.multinomial(torch.exp(x_log_probs[:,i]), 1).to(self.device)
            # update the batch with new samples
            x_seq = np.concatenate((x_seq, x_i_sample.to('cpu').detach().numpy()), 1)

        return x_seq

    @torch.no_grad()
    def top1_rec(self, x, causal=True):
        x_prob = torch.exp(self.transformer_forward(x, causal=True))[:,:-1,:].contiguous()
        _, x_rec_max = torch.max(x_prob, dim=2)
        return torch.sum(torch.mean((x_rec_max.float() == x[:,1:].float().to(device)).float(), 1).float())

    def forward(self, x, causal=True, temperature=1.0, reduction='mean'):
        # get log-probabilities
        log_prob = self.transformer_forward(x, causal=causal, temperature=temperature)

        return self.loss_fun(log_prob[:,:-1].contiguous(), x[:,1:].contiguous(), reduction=reduction)

### Evaluation and training functions

**Please DO NOT remove or modify them.**

In [ ]:
# DO NOT REMOVE OR MODIFY
def evaluation(test_loader, name=None, model_best=None, epoch=None, device='cuda'):
    # EVALUATION
    if model_best is None:
        # load best performing model
        model_best = torch.load(name + '.model').to(device)

    model_best.eval()
    loss = 0.
    rec = 1.
    N = 0.
    for indx_batch, test_batch in enumerate(test_loader):
        loss_t = model_best.forward(test_batch.to(device), reduction='sum')
        loss = loss + loss_t.item()

        rec_t = model_best.top1_rec(test_batch.to(device))
        rec = rec + rec_t.item()

        N = N + test_batch.shape[0]
    loss = loss / N
    rec = rec / N

    if epoch is None:
        print(f'FINAL LOSS: nll={loss}, rec={rec}')
    else:
        print(f'Epoch: {epoch}, val nll={loss}, val rec={rec}')

    return loss, rec

def plot_curve(name, nll_val, ylabel='nll'):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel('epochs')
    plt.ylabel(ylabel)
    plt.savefig(name + '_' + ylabel + '_val_curve.pdf', bbox_inches='tight')
    plt.close()

In [ ]:
# DO NOT REMOVE OR MODIFY
def training(name, max_patience, num_epochs, model, optimizer, training_loader, val_loader, device='cuda'):
    nll_val = []
    rec_val = []
    best_nll = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # TRAINING
        model.train()
        for indx_batch, batch in enumerate(training_loader):
            loss = model.forward(batch.to(device))

            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()

        # Validation
        loss_val, r_val = evaluation(val_loader, model_best=model, epoch=e, device=device)
        nll_val.append(loss_val)  # save for plotting
        rec_val.append(r_val)

        if e == 0:
            print('saved!')
            torch.save(model, name + '.model')
            best_nll = loss_val

            sampled_tokens = model.sample(batch_size=64, temperature=1.0)
            sampled_texts = tokenizer.decode(sampled_tokens)
            save_texts(sampled_texts, name='epoch_' + str(e))

        else:
            if loss_val < best_nll:
                print('saved!')
                torch.save(model, name + '.model')
                best_nll = loss_val
                patience = 0

                sampled_tokens = model.sample(batch_size=64, temperature=1.0)
                sampled_texts = tokenizer.decode(sampled_tokens)
                save_texts(sampled_texts, name='epoch_' + str(e))
            else:
                patience = patience + 1

        if patience > max_patience:
            break

    nll_val = np.asarray(nll_val)
    rec_val = np.asarray(rec_val)

    np.save(name + '_nll_val.npy', nll_val)
    np.save(name + '_rec_val.npy', rec_val)

    return nll_val, rec_val

### Setup

**NOTE: *Please comment your code! Especially if you introduce any new variables (e.g., hyperparameters).***

In [ ]:
# DO NOT REMOVE OR MODIFY
dataprocessor = DataProcessor()
tokenizer = Tokenizer(max_length=149)

In [ ]:
# PLEASE MODIFY ACCORDING TO THE REPORT REQUIREMENTS
num_training_data = 1000  # None to take all training data

# DO NOT REMOVE OR MODIFY THE REST OF THIS CELL
#-dataset
train_dataset = Headers(dataprocessor, tokenizer, num_training_data=num_training_data, mode="train")
validation_dataset = Headers(dataprocessor, tokenizer, mode="val")
test_dataset = Headers(dataprocessor, tokenizer, mode="test")

#-dataloaders
BATCH_SIZE = 32

training_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# DO NOT REMOVE (but you can modify if necessary)
#-creating a dir for saving results
name = 'arm_transformer'  # NOTE: if you run multiple experiments, you would overwrite results. Please modify this part if necessary.
results_dir = results_model_dir + name + '/'
if not(os.path.exists(results_dir)):
  os.mkdir(results_dir)

In the next cell, please initialize the model. Please remember about commenting your code!

In [ ]:
# DO NOT REMOVE but PLEASE MODIFY WHENEVER YOU ARE ASKED FOR IT!
# NOTE: in order to obtain required sizes of your models, you can play with
#       various values of num_neurons, num_heads, num_blocks, num_emb
num_tokens = 150 # do not modify!
num_token_vals = 29  # do not modify!
num_neurons = 16 # please modify it
num_heads = 4 # please modify it
num_blocks = 4 # please modify it
num_emb = num_heads * 4  # please modify it but it must be a multiplication of num_heads
causal=True # do not modify!

lr = 1e-3 # learning rate; do not modify!
num_epochs = 1000 # max. number of epochs; do not modify!
max_patience = 10 # an early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped; do not modify!

In [ ]:
# DO NOT REMOVE OR MODIFY
model = DecoderTransformer(num_tokens=num_tokens, num_token_vals=num_token_vals, num_emb=num_emb, num_neurons=num_neurons, num_heads=num_heads, num_blocks=num_blocks, device=device)
model = model.to(device)
# Print the summary (like in Keras)
print(summary(model, torch.zeros(1, num_tokens, dtype=torch.long).to(device), show_input=False, show_hierarchical=False))

Please initialize the optimizer

In [ ]:
# DO NOT REMOVE OR MODIFY
optimizer = torch.optim.AdamW([p for p in model.parameters() if p.requires_grad == True], lr=lr)

### Training and final evaluation

In the following two cells, we run the training and the final evaluation.

In [ ]:
# DO NOT REMOVE OR MODIFY
# Training procedure
nll_val, rec_val = training(name=results_dir + name, max_patience=max_patience, num_epochs=num_epochs, model=model, optimizer=optimizer, training_loader=training_loader, val_loader=val_loader, device=device)

In [ ]:
# DO NOT REMOVE OR MODIFY
# Final evaluation
test_loss, test_rec = evaluation(name=results_dir + name, test_loader=, device=device)

with open(results_dir + name + '_test_loss.txt', "w") as f:
    f.write('Test NLL: ' + str(test_loss)+'\n'+'Test REC: ' + str(test_rec))
    f.close()

plot_curve(results_dir + name, nll_val, ylabel='nll')
plot_curve(results_dir + name, rec_val, ylabel='rec')

In [ ]:
# DO NOT REMOVE
# Sample texts: load best model
model_best = torch.load(results_dir + name + '.model')
model_best = model_best.eval()

# sample
temperature = 1.0 # you can modify it
num_samples = 64 # you can modify it

sampled_tokens = model_best.sample(batch_size=num_samples, temperature=temperature)  # do not modify
sampled_texts = tokenizer.decode(sampled_tokens)  # do not modify

save_texts(sampled_texts, name='FINAL_' + str(temperature))